In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [22]:
history = 10
ac_pitch_window = sorted([-19, -12, 0, 12, 19])
la_pitch_window = list(range(-5, 6))
num_features = 7 #X.shape[1] - len(ac_pitch_window) * history - history * len(la_pitch_window) - 1

acoustic_in = keras.layers.Input(shape=(len(ac_pitch_window) * history,), dtype='float', name='acoustic')
language_in = keras.layers.Input(shape=(len(la_pitch_window) * history,), dtype='float', name='language')
features_in = keras.layers.Input(shape=(num_features + 1,), dtype='float', name='features')

# Acoustic model history - 1d convolutions in each direction
acoustic = keras.layers.Reshape((len(ac_pitch_window), history, 1),
                                input_shape=(len(ac_pitch_window) * history,))(acoustic_in)
ac_num_pitch_convs = 5
ac_num_history_convs = 10
acoustic = keras.layers.Conv2D(ac_num_pitch_convs, (len(ac_pitch_window), 1))(acoustic)
acoustic = keras.layers.Permute((3, 2, 1))(acoustic)
acoustic = keras.layers.Conv2D(ac_num_history_convs, (ac_num_pitch_convs, min(history, 5)))(acoustic)
acoustic = keras.layers.Flatten()(acoustic)

# Language model history - series of 2D convolutions
language = keras.layers.Reshape((len(la_pitch_window), history, 1),
                                input_shape=(len(la_pitch_window) * history,))(language_in)
language = keras.layers.Conv2D(5, (3, 3), strides=1)(language)
language = keras.layers.Conv2D(5, (3, 3), strides=2)(language)
language = keras.layers.Flatten()(language)

# Dense layers
dense_in = keras.layers.Concatenate()([acoustic, language, features_in])

dense = keras.layers.Dense(20, activation='relu')(dense_in)
dense = keras.layers.Dropout(0.2)(dense)

dense = keras.layers.Dense(20, activation='relu')(dense)
dense = keras.layers.Dropout(0.2)(dense)

dense = keras.layers.Dense(20, activation='relu')(dense)
dense = keras.layers.Dropout(0.2)(dense)

dense_out = keras.layers.Dense(1, activation='sigmoid')(dense)

model = keras.models.Model(inputs=[acoustic_in, language_in, features_in], outputs=dense_out)


In [23]:
X = np.random.rand(10000, len(ac_pitch_window) * history + history * len(la_pitch_window) + 1 + 7)
Y = np.round(np.random.rand(10000))

acoustic_in = X[:, :len(ac_pitch_window) * history]
language_in = X[:, len(ac_pitch_window) * history:history * (len(ac_pitch_window) + len(la_pitch_window))]
features_in = X[:, history * (len(ac_pitch_window) + len(la_pitch_window)):]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit([acoustic_in, language_in, features_in], Y, epochs=100, batch_size=32)


Epoch 1/100
10000/10000 [==============================] - 2s 195us/step - loss: 0.6942 - acc: 0.5079
Epoch 2/100
10000/10000 [==============================] - 1s 124us/step - loss: 0.6934 - acc: 0.5005
Epoch 3/100
10000/10000 [==============================] - 1s 125us/step - loss: 0.6932 - acc: 0.4997
Epoch 4/100
10000/10000 [==============================] - 1s 126us/step - loss: 0.6928 - acc: 0.5098
Epoch 5/100
10000/10000 [==============================] - 1s 126us/step - loss: 0.6928 - acc: 0.5069
Epoch 6/100
10000/10000 [==============================] - 1s 125us/step - loss: 0.6934 - acc: 0.5063
Epoch 7/100
10000/10000 [==============================] - 1s 127us/step - loss: 0.6928 - acc: 0.5127
Epoch 8/100
10000/10000 [==============================] - 1s 126us/step - loss: 0.6917 - acc: 0.5185
Epoch 9/100
10000/10000 [==============================] - 1s 127us/step - loss: 0.6897 - acc: 0.5271
Epoch 10/100
10000/10000 [==============================] - 1s 128us/step - loss: 

10000/10000 [==============================] - 1s 127us/step - loss: 0.5787 - acc: 0.6769
Epoch 81/100
10000/10000 [==============================] - 1s 127us/step - loss: 0.5744 - acc: 0.6768
Epoch 82/100
10000/10000 [==============================] - 1s 127us/step - loss: 0.5716 - acc: 0.6791
Epoch 83/100
10000/10000 [==============================] - 1s 129us/step - loss: 0.5697 - acc: 0.6836
Epoch 84/100
10000/10000 [==============================] - 1s 125us/step - loss: 0.5692 - acc: 0.6809
Epoch 85/100
10000/10000 [==============================] - 1s 126us/step - loss: 0.5691 - acc: 0.6850
Epoch 86/100
10000/10000 [==============================] - 1s 125us/step - loss: 0.5705 - acc: 0.6773
Epoch 87/100
10000/10000 [==============================] - 1s 126us/step - loss: 0.5683 - acc: 0.6827
Epoch 88/100
10000/10000 [==============================] - 1s 125us/step - loss: 0.5718 - acc: 0.6763
Epoch 89/100
10000/10000 [==============================] - 1s 126us/step - loss: 0.56